In [187]:
import dare
import random
import time
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from SISA_lib import SISA

In [188]:
data = pd.read_csv("cleaned_loan.csv")
data

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung,Keputusan Akhir
0,KARYAWAN,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (< 514,750,000)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),"≥ 2, < 5",SINGLE : ≥RP 10 JUTA,...,>= 15%,DEPOSITO/INVES/EMAS ≥ LIMIT KREDIT,"Keuangan, asuransi, perbankan",Cakupan area : nasional,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,READY STOK,GREEN,LAYAK,Diterima
1,PENGUSAHA,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,PISAH HARTA,36 - 45,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,DEPOSITO/INVES/EMAS ≥ LIMIT KREDIT,Lainnya,Cakupan area : nasional,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,READY STOK,GREEN,LAYAK,Diterima
2,KARYAWAN,"TAKE OVER + TOP UP KPR, RUMAH TINGGAL","Plafon (> 1,340,960,436)",JOINT INCOME,TIDAK PISAH HARTA,46 - 55,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,PROPERTI/KENDARAAN ≥ LIMIT KREDIT,KLINIK AKUPUNTIR,CAKUPAN AREA : LOKAL,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,SEDANG DIHUNI,GREEN,LAYAK,Diterima
3,PENGUSAHA,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (894,632,536 - 1,340,960,436)",JOINT INCOME,TIDAK PISAH HARTA,36 - 45,"Kol 1, Tidak pernah menunggak",RUMAH TINGGAL (RUMAH KE 2/3),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,PROPERTI/KENDARAAN < 100% LIMIT KREDIT,PERDAGANGAN,CAKUPAN AREA : LOKAL,"SHM, SERTIPIKAT PECAH",RUKO/RUKAN,READY STOK,GREEN,LAYAK,Diterima
4,KARYAWAN,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (< 514,750,000)",JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),"≥ 2, < 5","SINGLE : ≥RP 6 JUTA, < RP. 10 JUTA",...,>= 15%,KAS DAN NON KAS ≥ LIMIT KREDIT,COLOUR SALON,CAKUPAN AREA : LOKAL,"SHGB, SERTIPIKAT PECAH",Rumah Tinggal,SEDANG DIHUNI,YELLOW,LAYAK,Diterima
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal > Tipe 70","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),"≥ 2, < 5",SINGLE : ≥RP 20 JUTA,...,>= 15%,Properti/kendaraan < 50% plafon,"INFORMASI, TELEKOMUNIKASI DAN KOMPUTER",Public listed,MOU - Sertipikat Induk,Rumah Tinggal,Indent,YELLOW,TIDAK,Ditolak
196,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal Tipe 2...","Plafon (< 514,750,000)",JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),≥ 5,"MENIKAH : ≥RP 10 JUTA, <RP 20 JUTA",...,>= 15%,DEPOSITO/INVES/EMAS < 50% plafon,Lainnya,Multinasional,"SHGB, SERTIPIKAT PECAH",Rumah Tinggal,Indent,GREEN,TIDAK,Ditolak
197,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal Tipe 2...","Plafon (< 514,750,000)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),"≥ 2, < 5","SINGLE : ≥RP 6 JUTA, < RP. 10 JUTA",...,>= 15%,DEPOSITO/INVES/EMAS < 50% plafon,Lainnya,CAKUPAN AREA : LOKAL,MOU - Sertipikat Induk,Rumah Tinggal,Indent,YELLOW,TIDAK,Ditolak
198,PENGUSAHA,"Pembelian Baru / Primary, Ruko / Rukan","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,PISAH HARTA,56 - 65,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,null < 50% plafon,Lainnya,Multinasional,MOU - Sertipikat Induk,RUKO/RUKAN,Indent,GREEN,TIDAK,Ditolak


In [189]:
labelEncoder = {}
for col in data.columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    labelEncoder[col] = le
data

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung,Keputusan Akhir
0,0,1,2,1,1,0,0,2,1,5,...,1,2,14,1,6,2,1,0,0,0
1,1,1,3,1,0,1,8,2,2,3,...,1,2,15,1,6,2,1,0,0,0
2,0,9,3,0,1,2,8,2,2,3,...,1,9,12,0,6,2,2,0,0,0
3,1,1,1,0,1,1,8,1,2,3,...,1,7,19,0,6,1,1,0,0,0
4,0,1,2,0,1,0,0,2,1,8,...,1,5,2,0,3,2,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,6,3,1,1,0,8,2,1,7,...,1,10,6,4,0,2,0,2,1,1
196,0,7,2,0,1,0,0,2,2,2,...,1,1,15,3,3,2,0,0,1,1
197,0,7,2,1,1,0,8,2,1,8,...,1,1,15,0,0,2,0,2,1,1
198,1,5,3,1,0,3,8,2,2,3,...,1,12,15,3,0,1,0,0,1,1


In [190]:
data = data.drop_duplicates()
data

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung,Keputusan Akhir
0,0,1,2,1,1,0,0,2,1,5,...,1,2,14,1,6,2,1,0,0,0
1,1,1,3,1,0,1,8,2,2,3,...,1,2,15,1,6,2,1,0,0,0
2,0,9,3,0,1,2,8,2,2,3,...,1,9,12,0,6,2,2,0,0,0
3,1,1,1,0,1,1,8,1,2,3,...,1,7,19,0,6,1,1,0,0,0
4,0,1,2,0,1,0,0,2,1,8,...,1,5,2,0,3,2,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,6,3,1,1,0,8,2,1,7,...,1,10,6,4,0,2,0,2,1,1
196,0,7,2,0,1,0,0,2,2,2,...,1,1,15,3,3,2,0,0,1,1
197,0,7,2,1,1,0,8,2,1,8,...,1,1,15,0,0,2,0,2,1,1
198,1,5,3,1,0,3,8,2,2,3,...,1,12,15,3,0,1,0,0,1,1


In [191]:
X = data.drop(["Keputusan Akhir"], axis=1)
y = data["Keputusan Akhir"]

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [193]:
X_train_base = X_train.copy()
y_train_base = y_train.copy()
X_test_base = X_test.copy()

In [194]:
#SISA train and test dataset formats
X_train_SISA = X_train.values
y_train_SISA = y_train.values
X_test_SISA = X_test.values

In [195]:
#DaRE Forest train dataset
X_train_dare = np.array(X_train.values.tolist())
y_train_dare = np.array(y_train.values.tolist())
X_test_dare = np.array(X_test.values.tolist())

In [196]:
#make model for each machine learning model

In [197]:
#Base Model using RandomForest
tree_model = RandomForestClassifier(n_estimators=50,max_depth=5, random_state=42)
tree_model.fit(X_train_base, y_train_base)

RandomForestClassifier(max_depth=5, n_estimators=50, random_state=42)

In [198]:
#predict using XGBoost model
y_pred_tree = tree_model.predict(X_test_base)
accuracy_tree = accuracy_score(y_pred_tree, y_test)
print(f"Akurasi Model Decision Tree : {accuracy_tree}")

Akurasi Model Decision Tree : 0.9722222222222222


In [199]:
#SISA Model akurasi by shard
shard = 2
accuracy = 1
tertinngi = 0
accuracy_tertinngi = 0
exp_results = pd.DataFrame(columns=['Shard', 'Accuracy'])

In [200]:
while shard <= 6:
    sisa_by_shard = SISA(shard)
    sisa_by_shard .train(X_train_SISA, y_train_SISA)
    y_pred = sisa_by_shard .predict(X_test_SISA)
    accuracy = accuracy_score(y_pred, y_test)
    exp_results = pd.concat([exp_results, pd.DataFrame([{'Shard': shard, 'Accuracy': accuracy}])], ignore_index=True)
    if (accuracy > accuracy_tertinngi):
        accuracy_tertinngi = accuracy
        tertinngi = shard
    shard += 1

In [201]:
#show sisa accuracy by shard
data_result = pd.DataFrame(exp_results)
data_result

,Shard,Accuracy
0,2,0.958333
1,3,0.958333
2,4,0.930556
3,5,0.958333
4,6,0.902778


In [202]:
print(tertinngi)

2


In [203]:
#Base on result below, SISA model will use 4 shard for expriment
sisa = SISA(3)
sisa.train(X_train_SISA, y_train_SISA)

In [204]:
y_pred_SISA = sisa.predict(X_test_SISA)
accuracy_SISA = accuracy_score(y_pred_SISA, y_test)
print(f"Akurasi Model SISA : {accuracy_SISA}")

Akurasi Model SISA : 0.9583333333333334


In [205]:
#DaRE Forest Model
dafo = dare.Forest(n_estimators=50, max_depth=5, k = 5, topd = 0, random_state=42)
dafo.fit(X_train_dare, y_train_dare)

In [206]:
#Predict using DaRE Forest Model
y_pred_dare = dafo.predict(X_test_dare)
accuracy_dare = accuracy_score(y_pred_dare, y_test)
print(f"Akurasi Model DaRE Forest : {accuracy_dare}")

Akurasi Model DaRE Forest : 0.9444444444444444


In [207]:
test_proba_dare = np.array(X_test_dare)
dafo.predict_proba(test_proba_dare)

array([[0.1479196 , 0.8520804 ],
       [0.99833333, 0.00166667],
       [0.05336842, 0.94663158],
       [0.9505    , 0.0495    ],
       [0.09635043, 0.90364957],
       [0.77707001, 0.22292999],
       [0.18791207, 0.81208793],
       [0.92486755, 0.07513245],
       [0.70715725, 0.29284275],
       [0.77185131, 0.22814869],
       [0.77311321, 0.22688679],
       [0.02332609, 0.97667391],
       [0.99547619, 0.00452381],
       [0.15041625, 0.84958375],
       [0.97352124, 0.02647876],
       [0.10802991, 0.89197009],
       [0.07336351, 0.92663649],
       [0.74023292, 0.25976708],
       [0.31166947, 0.68833053],
       [0.14225342, 0.85774658],
       [0.77792715, 0.22207285],
       [0.6512146 , 0.3487854 ],
       [0.08636985, 0.91363015],
       [0.23282102, 0.76717898],
       [0.96980952, 0.03019048],
       [0.24545301, 0.75454699],
       [0.12520472, 0.87479528],
       [0.07723341, 0.92276659],
       [0.10315004, 0.89684996],
       [0.09851167, 0.90148833],
       [0.

In [208]:
test_proba_base = np.array(X_test_base)
tree_model.predict_proba(test_proba_base)

c:\Users\pilha\anaconda3\envs\TA\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([[0.18944866, 0.81055134],
       [0.9778744 , 0.0221256 ],
       [0.0802081 , 0.9197919 ],
       [0.96650466, 0.03349534],
       [0.0777129 , 0.9222871 ],
       [0.56189032, 0.43810968],
       [0.15789788, 0.84210212],
       [0.95602869, 0.04397131],
       [0.56247676, 0.43752324],
       [0.71052319, 0.28947681],
       [0.68874684, 0.31125316],
       [0.04125153, 0.95874847],
       [0.99714286, 0.00285714],
       [0.16265834, 0.83734166],
       [0.964516  , 0.035484  ],
       [0.18474189, 0.81525811],
       [0.14995584, 0.85004416],
       [0.7839862 , 0.2160138 ],
       [0.24589878, 0.75410122],
       [0.24966641, 0.75033359],
       [0.63200143, 0.36799857],
       [0.631127  , 0.368873  ],
       [0.09139555, 0.90860445],
       [0.15745068, 0.84254932],
       [0.95332528, 0.04667472],
       [0.29394102, 0.70605898],
       [0.14268675, 0.85731325],
       [0.15630031, 0.84369969],
       [0.08806222, 0.91193778],
       [0.1150381 , 0.8849619 ],
       [0.

In [209]:
#Perbandingan label untuk masing-masing model

In [210]:
comparation_pred_model = pd.DataFrame({
    'Decision Tree': y_pred_tree,
    'SISA': y_pred_SISA,
    'DaRE Forest': y_pred_dare
})

In [211]:
comparation_pred_model

,Decision Tree,SISA,DaRE Forest
0,1,1,1
1,0,0,0
2,1,1,1
3,0,0,0
4,1,1,1
...,...,...,...
67,0,0,0
68,0,0,0
69,1,1,1
70,1,1,1


#### Melakukan proses penghapusan secara random untuk masing-masing model

In [212]:
#Create dataFrame for each model
base_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy', 'Time'])
SISA_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy', 'Time'])
dare_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy', 'Time'])
comparition_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy Base', 'Accuracy SISA', 'Accuracy DaRE','Time Base', 'Time SISA', 'Time DaRE'])

In [213]:
#Create function to delete base model data
def delete_base_data(X_train_base, y_train_base, index): 
    X_data = X_train.iloc[index].tolist()
    data_to_delete = X_train_base.apply(lambda row: all(elem in row.values for elem in X_data), axis=1)
    X_train_base = X_train_base[~data_to_delete]
    y_train_base = y_train_base[~data_to_delete]
    return X_train_base, y_train_base

In [214]:
#create variabel for process
deleted_index = []
jumlah_data = len(X_train)
max_random_number = len(X_train) - 1
y_pred_comparison_flag = 0

In [215]:
while len(deleted_index) < max_random_number:
    #get new index
    new_index = False
    while new_index == False:
        index = random.randint(0, max_random_number - 1)
        if index not in deleted_index:
            deleted_index.append(index)
            break
    jumlah_data -= 1
    #Base Model with decreasing dataset
    try:
        start_time_base = time.time()
        X_train_base, y_train_base = delete_base_data(X_train_base, y_train_base,index)
        tree_model.fit(X_train_base, y_train_base)
        training_time_base = time.time() - start_time_base

        #prediction using base model with new data
        y_pred_tree = tree_model.predict(X_test_base)
        accuracy_tree = accuracy_score(y_pred_tree, y_test)

        #add to dataframe
        base_model = pd.concat([base_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': accuracy_tree, "Time" : training_time_base}])], ignore_index=True)
    except:
        print("Penghapusan data mencapai maksimal")
        break

    #delete elemen by sisa
    try:
        start_time_sisa = time.time()
        sisa.delete([X_train_SISA[index]])
        training_time_sisa = time.time() - start_time_sisa

        #prediction using model with deleted data
        y_pred_SISA = sisa.predict(X_test_SISA)
        accuracy_SISA = accuracy_score(y_pred_SISA, y_test)

        #add to dataframe
        SISA_model = pd.concat([SISA_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': accuracy_SISA, "Time" : training_time_sisa}])], ignore_index=True)
    except:
        print("salah satu data pada shard, sudah habis, silahkan train ulang model dari awal")
        # SISA_model = pd.concat([SISA_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': "-", "Time" : "-"}])], ignore_index=True)
        accuracy_SISA = "-"
        training_time_sisa = "-"
    
    #delete elemen by dareForest
    try:
        start_time_dare = time.time()
        dafo.delete(index)
        training_time_dare = time.time() - start_time_dare

        #prediction using model with deleted data
        y_pred_dare = dafo.predict(X_test_dare)
        accuracy_dare = accuracy_score(y_pred_dare, y_test)

        #add to dataframe
        dare_model = pd.concat([dare_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': accuracy_dare, "Time" : training_time_dare}])], ignore_index=True)
    except Exception as e:
        print(e)

    #comparisson model
    comparition_model = pd.concat([comparition_model, pd.DataFrame([{
        'Jumlah data' : jumlah_data,
        'Accuracy Base' : accuracy_tree,
        'Accuracy SISA' : accuracy_SISA,
        'Accuracy DaRE' : accuracy_dare,
        'Time Base' : training_time_base,
        'Time SISA' : training_time_sisa,
        'Time DaRE' : training_time_dare,
    }])], ignore_index=True)

    #comparisoon y_pred for each model
    if accuracy_tree < 0.7 and y_pred_comparison_flag == 0:
        y_pred_comparison_flag = 1
        y_pred_comparation = pd.DataFrame({
            'Data ke -': jumlah_data,
            'Decision Tree': y_pred_tree,
            'SISA': y_pred_SISA,
            'DaRE Forest': y_pred_dare,
            'y_test' : y_test
        })


salah satu data pada shard, sudah habis, silahkan train ulang model dari awal
salah satu data pada shard, sudah habis, silahkan train ulang model dari awal
Penghapusan data mencapai maksimal


In [216]:
comparition_model.to_csv("hasil_eksperiment/exp_3/hasil_eksperiment.csv", index=False)

In [217]:
y_pred_comparation.to_csv("hasil_eksperiment/exp_3/y_pred_compare.csv", index=False)

In [218]:
X_train

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Jangka Waktu Kredit,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung
27,0,6,3,1,1,0,8,2,2,7,...,0,0,10,15,1,0,2,0,0,1
120,0,4,2,1,1,1,8,2,1,1,...,1,1,11,21,1,0,0,4,2,0
4,0,1,2,0,1,0,0,2,1,8,...,2,1,5,2,0,3,2,2,2,0
32,1,6,3,0,1,2,7,2,2,3,...,0,1,12,15,0,0,2,4,0,1
109,0,4,2,1,1,0,8,2,1,7,...,1,1,11,15,0,0,0,4,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,1,1,0,0,1,0,8,2,1,3,...,0,1,5,6,0,1,2,0,0,0
114,0,4,0,1,1,2,8,2,2,3,...,1,1,12,14,1,0,0,4,2,0
14,0,4,0,1,1,0,8,2,1,5,...,0,0,10,15,0,0,0,4,2,0
100,1,5,1,0,1,1,0,1,2,3,...,0,1,11,13,1,0,1,0,0,0
